# Fortran vs Julia

I  this notebook I intend to compare the Fortran and Julia solvers for the PS3 model.

First we will compare both solvers in regards to solving the Value Function.

In [4]:
using Distributed, SharedArrays, BenchmarkTools
cd("/home/mitchv34/Work/2nd Year/ECON 899 (Computational Methods)/1st Quarter/Problem Sets/Shared Repo/Shared Repo")

In [8]:
include("../JuliaCode/conesa_kueger.jl");

In [17]:
# This funciton will compile and run the Fortran code
function V_fun_Fortran()
    path = "/home/mitchv34/Work/2nd Year/ECON 899 (Computational Methods)/1st Quarter/Problem Sets/Shared Repo/Shared Repo/PS3/FortranCode/"
    run(`gfortran -fopenmp -O2 -o $(path)V_fun_Fortran $(path)conesa_kueger.f90`)
    run(`$(path)V_Fortran`)
end

V_fun_Fortran (generic function with 1 method)

In [20]:
# We can benchmark the Fortran code
@benchmark V_fun_Fortran()

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.422 s …    1.775 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.695 s               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.647 s ± 157.521 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                     █            █     █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█ ▁
  1.42 s         Histogram: frequency by time         1.77 s <

 Memory estimate: 7.62 KiB, allocs estimate: 166.

In [21]:
# We can also benchmark the Julia code
@benchmark begin
    prim, res = Initialize()
    V_ret(prim, res)
    V_workers(prim, res)
end

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.825 s …   1.911 s  ┊ GC (min … max): 0.70% … 0.86%
 Time  (median):     1.861 s              ┊ GC (median):    0.69%
 Time  (mean ± σ):   1.865 s ± 43.327 ms  ┊ GC (mean ± σ):  0.75% ± 0.10%

  █                      █                                █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.82 s         Histogram: frequency by time        1.91 s <

 Memory estimate: 312.40 MiB, allocs estimate: 44575.

Being fair just runing the fortran code will not help a lot, next we will use a function that reads in the data from the Fortran code in a format that is easier to work with.

In [28]:
function V_fun_Fortran2(prim::Primitives, res::Results)
    @unpack r, w, b =res
    # PS3/FortranCode/conesa_kueger.f90
    # Compile Fortran code
    path = "/home/mitchv34/Work/2nd Year/ECON 899 (Computational Methods)/1st Quarter/Problem Sets/Shared Repo/Shared Repo/PS3/FortranCode/"
    run(`gfortran -fopenmp -O2 -o $(path)V_Fortran $(path)conesa_kueger.f90`)
    # run(`./T_op $q $n_iter`)
    run(`$(path)V_Fortran`)

    results_raw =  readdlm("$(path)results.csv");

    val_fun = zeros(prim.nA, prim.nZ, prim.N_final)    # Initialize the value function
    pol_fun = zeros(prim.nA, prim.nZ, prim.N_final)    # Initialize the policy function
    pol_fun_ind = zeros(prim.nA, prim.nZ, prim.N_final + 1)    # Initialize the policy function index
    consumption = zeros(prim.nA, prim.nZ, prim.N_final)    # Initialize the consumption function
    l_fun = zeros(prim.nA, prim.nZ, prim.N_final)    # Initialize the labor policy function
    for j in 1:prim.N_final
        range_a = (j-1) * 2*prim.nA + 1 : j * 2*prim.nA |> collect
        val_fun[:,:,j] = hcat(results_raw[range_a[1:prim.nA],end], results_raw[range_a[prim.nA+1:end],end])
        pol_fun_ind[:,:,j] = hcat(results_raw[range_a[1:prim.nA],end-1], results_raw[range_a[prim.nA+1:end],end-1])
        pol_fun[:,:,j] = hcat(results_raw[range_a[1:prim.nA],end-2], results_raw[range_a[prim.nA+1:end],end-2])
        consumption[:,:,j]   = hcat(results_raw[range_a[1:prim.nA],end-3], results_raw[range_a[prim.nA+1:end],end-3])
        l_fun[:,:,j] = hcat(results_raw[range_a[1:prim.nA],end-4], results_raw[range_a[prim.nA+1:end],end-4])
    end
    A_grid_fortran = results_raw[1:prim.nA,end-5]
    res.val_fun = val_fun
    res.pol_fun = pol_fun
    res.pol_fun_ind = pol_fun_ind
    res.l_fun = l_fun
    return A_grid_fortran, consumption

end # run_Fortran()


V_fun_Fortran2 (generic function with 1 method)

In [34]:
@benchmark begin
    prim, res = Initialize()
    V_fun_Fortran2(prim, res)
end

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.882 s …   1.955 s  ┊ GC (min … max): 6.95% … 1.13%
 Time  (median):     1.917 s              ┊ GC (median):    1.15%
 Time  (mean ± σ):   1.918 s ± 36.259 ms  ┊ GC (mean ± σ):  2.77% ± 3.61%

  █                          █                            █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.88 s         Histogram: frequency by time        1.95 s <

 Memory estimate: 142.82 MiB, allocs estimate: 3169629.

But Fortran code is actually doing more than just solving the Value Function. It also solves for the stationary distribution and the aggregate levels of capital and labor.

Les's compare the results with a Julia code that does the same.

In [36]:
@benchmark begin
    prim, res = Initialize()
    
    V_ret(prim, res);
    V_workers(prim, res);

    SteadyStateDist(prim, res);

    # calculate aggregate capital and labor
    K = sum(res.F[:, :, :] .* prim.a_grid)
    L = sum(res.F[:, :, :] .* res.l_fun) # Labor supply grid
end

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.799 s …   2.866 s  ┊ GC (min … max): 2.75% … 2.89%
 Time  (median):     2.832 s              ┊ GC (median):    2.82%
 Time  (mean ± σ):   2.832 s ± 46.813 ms  ┊ GC (mean ± σ):  2.82% ± 0.09%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.8 s          Histogram: frequency by time        2.87 s <

 Memory estimate: 1.33 GiB, allocs estimate: 1925921.

This means that we can use Fortran to solve the model usign Fortran just with the aggreates which will reduce the computational load of reading in the data.

In [42]:
function solve_model_Fortran(prim, res; λ::Float64=0.7, tol::Float64=1e-2, err::Float64=100.0)
    prim, res= Initialize()
    @unpack w_mkt, r_mkt, b_mkt, J_R, a_grid = prim
    path = "/home/mitchv34/Work/2nd Year/ECON 899 (Computational Methods)/1st Quarter/Problem Sets/Shared Repo/Shared Repo/PS3/FortranCode/"
    
    n = 0 # loop counter
    while err > tol

        # calculate prices and payments at current K, L, and F
        res.r = r_mkt(res.K, res.L)
        res.w = w_mkt(res.K, res.L)
        res.b = b_mkt(res.L, res.w, sum(res.μ[J_R:end]))

        run(`gfortran -fopenmp -O2 -o $(path)V_Fortran $(path)conesa_kueger.f90`)
        # run(`./T_op $q $n_iter`)
        run(`$(path)V_Fortran $0 $(res.r) $(res.w) $(res.b)`)

        K, L = readdlm("$(path)agg_results.csv");

        # calculate error
        err = maximum(abs.([res.K, res.L] - [K, L]))

        if (err > tol*10)
            # Leave λ at the default
        elseif (err > tol*5) & (λ <= 0.85)
            λ = 0.85
        elseif (err > tol*1.01) & (λ <= 0.90)
            λ = 0.90
        elseif λ <= 0.95
            λ = 0.95
        end

        # update guess
        res.K = (1-λ)*K + λ*res.K
        res.L = (1-λ)*L + λ*res.L

        n+=1

        println("$n iterations; err = $err, K = ", round(res.K, digits = 4), ", L = ",
        round(res.L, digits = 4), ", λ = $λ")

    end # while err > tol
    # return prim, res
end

solve_model_Fortran (generic function with 1 method)

In [47]:
@time begin
    prim, res = Initialize()
    solve_model_Fortran(prim, res);
end

1 iterations; err = 1.853654283196522, K = 4.5561, L = 0.7164, λ = 0.7


2 iterations; err = 0.628222627598225, K = 4.3676, L = 0.5981, λ = 0.7


3 iterations; err = 0.7490854426911806, K = 4.1429, L = 0.5175, λ = 0.7


4 iterations; err = 0.6394546435929285, K = 3.9511, L = 0.4625, λ = 0.7


5 iterations; err = 0.5252353649988502, K = 3.7935, L = 0.4248, λ = 0.7


...

25 iterations; err = 0.014489272438640288, K = 3.3764, L = 0.3452, λ = 0.9


26 iterations; err = 0.013008477086864367, K = 3.3751, L = 0.345, λ = 0.9


27 iterations; err = 0.011832110795989781, K = 3.3739, L = 0.3448, λ = 0.9


28 iterations; err = 0.010596875039666998, K = 3.3729, L = 0.3447, λ = 0.9


29 iterations; err = 0.00958704319094128, K = 3.3724, L = 0.3446, λ = 0.95
 46.302382 seconds (14.30 k allocations: 6.037 MiB, 0.03% gc time)


Compared with the same code using Julia functions:

In [45]:
function MarketClearing_Julia(prim, res, λ::Float64=0.7, tol::Float64=1e-2, err::Float64=100.0)

    # unpack relevant variables and functions
    @unpack w_mkt, r_mkt, b_mkt, J_R, a_grid = prim

    n = 0 # loop counter

    # iteratively solve the model until excess savings converge to zero
    while err > tol

        # calculate prices and payments at current K, L, and F
        res.r = r_mkt(res.K, res.L)
        res.w = w_mkt(res.K, res.L)
        res.b = b_mkt(res.L, res.w, sum(res.μ[J_R:end]))

        # solve model with current model and payments
    
        V_ret(prim, res);
        V_workers(prim, res);

        SteadyStateDist(prim, res);

        # calculate aggregate capital and labor
        K = sum(res.F[:, :, :] .* a_grid)
        L = sum(res.F[:, :, :] .* res.l_fun) # Labor supply grid

        # calculate error
        err = maximum(abs.([res.K, res.L] - [K, L]))

        if (err > tol*10)
            # Leave λ at the default
        elseif (err > tol*5) & (λ <= 0.85)
            λ = 0.85
        elseif (err > tol*1.01) & (λ <= 0.90)
            λ = 0.90
        elseif λ <= 0.95
            λ = 0.95
        end

        # update guess
        res.K = (1-λ)*K + λ*res.K
        res.L = (1-λ)*L + λ*res.L

        n+=1

        println("$n iterations; err = $err, K = ", round(res.K, digits = 4), ", L = ",
        round(res.L, digits = 4), ", λ = $λ")

    end # while err > tol
    # return prim, res
end # MarketClearing

MarketClearing_Julia (generic function with 4 methods)

In [48]:
@time begin
    prim, res = Initialize()
    solve_model_Fortran(prim, res);
end

1 iterations; err = 1.853654283196522, K = 4.5561, L = 0.7164, λ = 0.7


2 iterations; err = 0.628222627598225, K = 4.3676, L = 0.5981, λ = 0.7


3 iterations; err = 0.7490854426911806, K = 4.1429, L = 0.5175, λ = 0.7


4 iterations; err = 0.6394546435929285, K = 3.9511, L = 0.4625, λ = 0.7


5 iterations; err = 0.5252353649988502, K = 3.7935, L = 0.4248, λ = 0.7


...

25 iterations; err = 0.014489272438640288, K = 3.3764, L = 0.3452, λ = 0.9


26 iterations; err = 0.013008477086864367, K = 3.3751, L = 0.345, λ = 0.9


27 iterations; err = 0.011832110795989781, K = 3.3739, L = 0.3448, λ = 0.9


28 iterations; err = 0.010596875039666998, K = 3.3729, L = 0.3447, λ = 0.9


29 iterations; err = 0.00958704319094128, K = 3.3724, L = 0.3446, λ = 0.95
 46.064710 seconds (14.26 k allocations: 6.034 MiB)


We can see that both converge in the same number of iterations to the same solution at virtually the same speed.

Finally we will use Fortran to solve the model completely.

In [52]:
function Fortran_sove_the_whole_thing()
    path = "/home/mitchv34/Work/2nd Year/ECON 899 (Computational Methods)/1st Quarter/Problem Sets/Shared Repo/Shared Repo/PS3/FortranCode/"
    run(`gfortran -fopenmp -O2 -o $(path)V_Fortran $(path)conesa_kueger.f90`)
    # run(`./T_op $q $n_iter`)
    run(`$(path)V_Fortran $1 $4 $0.9`)

    results_raw =  readdlm("$(path)results.csv");

    val_fun = zeros(prim.nA, prim.nZ, prim.N_final)    # Initialize the value function
    pol_fun = zeros(prim.nA, prim.nZ, prim.N_final)    # Initialize the policy function
    pol_fun_ind = zeros(prim.nA, prim.nZ, prim.N_final + 1)    # Initialize the policy function index
    consumption = zeros(prim.nA, prim.nZ, prim.N_final)    # Initialize the consumption function
    l_fun = zeros(prim.nA, prim.nZ, prim.N_final)    # Initialize the labor policy function
    for j in 1:prim.N_final
        range_a = (j-1) * 2*prim.nA + 1 : j * 2*prim.nA |> collect
        val_fun[:,:,j] = hcat(results_raw[range_a[1:prim.nA],end], results_raw[range_a[prim.nA+1:end],end])
        pol_fun_ind[:,:,j] = hcat(results_raw[range_a[1:prim.nA],end-1], results_raw[range_a[prim.nA+1:end],end-1])
        pol_fun[:,:,j] = hcat(results_raw[range_a[1:prim.nA],end-2], results_raw[range_a[prim.nA+1:end],end-2])
        consumption[:,:,j]   = hcat(results_raw[range_a[1:prim.nA],end-3], results_raw[range_a[prim.nA+1:end],end-3])
        l_fun[:,:,j] = hcat(results_raw[range_a[1:prim.nA],end-4], results_raw[range_a[prim.nA+1:end],end-4])
    end
    A_grid_fortran = results_raw[1:prim.nA,end-5]
    res.val_fun = val_fun
    res.pol_fun = pol_fun
    res.pol_fun_ind = pol_fun_ind
    res.l_fun = l_fun

    return nothing
end 

Fortran_sove_the_whole_thing (generic function with 1 method)

In [54]:
@time begin
    Fortran_sove_the_whole_thing();
end

 Iteration:           0 ERR=   1.9040204465183033      K=   4.2856030215823608      L=  0.80821288785858592      LAMBDA=  0.85000002384185791     


 Iteration:           1 ERR=   1.0099861783401369      K=   4.4371009242534347      L=  0.73361572785500595      LAMBDA=  0.85000002384185791     


 Iteration:           2 ERR=  0.42798928423478255      K=   4.5012993066845919      L=  0.67234078960674060      LAMBDA=  0.85000002384185791     


 Iteration:           3 ERR=  0.33920737965243147      K=   4.5304648847239344      L=  0.62145969074621010      LAMBDA=  0.85000002384185791     


 Iteration:           4 ERR=  0.28289431902224044      K=   4.5308357670169617      L=  0.57902554963760022      LAMBDA=  0.85000002384185791     


...

 Iteration:          28 ERR=   1.7795482906533877E-002 K=   4.1912204500217403      L=  0.36732539295246452      LAMBDA=  0.89999997615814209     


 Iteration:          29 ERR=   1.6398700802177757E-002 K=   4.1895805795505474      L=  0.36678758420362323      LAMBDA=  0.89999997615814209     


 Iteration:          30 ERR=   5.2032612827401792E-003 K=   4.1893204164243825      L=  0.36655609161406699      LAMBDA=  0.94999998807907104     
 Total elapsed time =    7.99200010      seconds


  9.812714 seconds (3.18 M allocations: 141.008 MiB, 0.09% gc time)


1000×2×66 Array{Float64, 3}:
[:, :, 1] =
 1.07313  0.125839
 1.07263  0.0935282
 1.07213  0.0930274
 1.07163  0.0925267
 1.07113  0.0920259
 1.07063  0.0597153
 1.03832  0.0592145
 1.03782  0.0587138
 1.03732  0.058213
 1.03681  0.0577122
 ⋮        
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0

[:, :, 2] =
 1.15486  0.134159
 1.15436  0.101848
 1.15386  0.101347
 1.15336  0.100847
 1.15286  0.100346
 1.15236  0.0680352
 1.15186  0.0675344
 1.11955  0.0670337
 1.11904  0.0665329
 1.11854  0.0660322
 ⋮        
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0

[:, :, 3] =
 1.23659  0.142479
 1.23609  0.110168
 1.23559  0.109667
 1.23509  0.109167
 1.23459  0.108666
 1.23409  0.0763551
 1.23359  0.0758544
 1.23308  0.0753536
 1.23258  0.0748528
 1.23208  0.0743521
 ⋮        
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0

Clearly we only get significant speedup from using Fortran to run the whole model.